In [1]:
%%capture
!pip install datasets

### Get SQL queries

In [3]:
from datasets import load_dataset

sql = load_dataset('wikisql', split='test')

Using custom data configuration default


Dataset wiki_sql downloaded and prepared to /root/.cache/huggingface/datasets/wiki_sql/default/0.1.0/be1740ee370ee37c17ff39d748e2440c7683b0d99ae93b8019b294d0113b6d90. Subsequent calls will reuse this data.


In [ ]:
sql = sql['sql'].sample(len(sql), random_state=1113)[:1000]
sql = [x['human_readable'] for x in sql]
print(sql[:10])
print(len(sql))

In [ ]:
train_sql = sql.copy()

In [ ]:
import pandas as pd

test_df = pd.DataFrame({'sql_sentence': test_sql})
test_df.to_csv('test_df.csv', index=False)

In [ ]:
pd.read_csv('test_df.csv').shape

(15878, 1)

### Get sentences

In [ ]:
sentences = load_dataset('glue', 'stsb', split='test')
print(sentences.shape)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


(1379, 4)


In [ ]:
sentences = sentences.shuffle(seed=1113)['sentence1'][:1000]
print(sentences[:10])

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/glue/stsb/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-b566652a2a364b60.arrow


['Storm buries Northeast; 16 inches of snow in NJ', "You don't have to know.", 'A girl is playing guitar.', "I don't think it makes any tremendous difference.", 'Militant attack from Afghanistan kills four in Upper Dir', 'The New York senator\'s new book, "Living History," appears a certain bestseller.', "I think it's fine to ask this question.", 'Schiphol airport partly evacuated after bomb threat', 'Syria fighting rages as Russia urges Western re-think', 'Nigeria mourns over 193 people killed in plane crash']


### Generate dataset

In [ ]:
import pandas as pd 
import numpy as np

data = pd.DataFrame({'sentence': sql+sentences, 'is_sql': np.concatenate((np.ones(1000), np.zeros(1000)))})
data.shape

(2000, 2)

In [ ]:
data.iloc[995:1005]

,sentence,is_sql
995,SELECT N° FROM table WHERE Television service ...,1.0
996,SELECT SUM Grid FROM table WHERE Laps = 30 AND...,1.0
997,SELECT Position FROM table WHERE Player = kenn...,1.0
998,SELECT Event FROM table WHERE Score points = 10,1.0
999,SELECT Date FROM table WHERE Opponent in the f...,1.0
1000,Storm buries Northeast; 16 inches of snow in NJ,0.0
1001,You don't have to know.,0.0
1002,A girl is playing guitar.,0.0
1003,I don't think it makes any tremendous difference.,0.0
1004,Militant attack from Afghanistan kills four in...,0.0


### Get embeddings

In [1]:
%%capture 
!pip install transformers

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
  
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenizer.save_pretrained('.')
model.save_pretrained('.')

In [ ]:
%%capture
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, models

word_embedding_model = models.Transformer('.', max_seq_length=15)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

In [ ]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
print(device)

cuda


In [ ]:
input = tokenizer(data['sentence'].tolist()[0], return_tensors="pt", padding=True).to(device)
output = model.encode(data['sentence'].tolist()[0])
outputs = torch.from_numpy(output)
outputs = outputs.unsqueeze(0)

for i in range(1, len(data)):
    output = model.encode(data['sentence'].tolist()[i])
    output = torch.from_numpy(output)
    output = output.unsqueeze(0)
    outputs = torch.cat((outputs, output), 0)

In [ ]:
outputs.shape

torch.Size([2000, 768])

In [ ]:
%%capture
pip install tensorboard

In [ ]:
!pip uninstall tensorflow

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter()
writer.add_embedding(outputs, metadata=data['is_sql'].tolist())